# 3D Target Tracking Optimization
## Author: Michel Barbeau, Carleton University
## Version: March 10, 2024


In [1]:
import numpy as np
import scipy.linalg as la
import matplotlib.pyplot as plt
import scipy.optimize as opt
from scipy.optimize import Bounds, LinearConstraint, NonlinearConstraint
from timeit import default_timer as timer
import math

### Target Positions

The traget positions are generated randomly, staring from start position $(x_0,y_0,z_0)$.

Position $(x_{i+1},y_{i+1},z_{i+1})$ is defined as $(x_i+\delta_x,y_i+\delta_y,z_i+\delta_z)$, where $\delta_x$, $\delta_y$, and $\delta_z$ are generated randomly in the interval $[-1,1]$.

The target positions are stored in array "pos" as the sequence $[x_0,x_1,\dots,x_{n-1},y_0,y_1,\dots,y_{n-1},z_0,z_1,\dots,z_{n-1}]$

### Objective function
The optimization object is to minimize the acceleration, defined as the difference of speed at time $i$ and $i-1$:
\begin{equation}
\vert s_{i+1} - s_{i} \vert \text{ for } i=0,\ldots,n-1.
\end{equation}
which is equal to the position differences:
\begin{equation}
\vert (x_{i+2} - x_{i+1}) - (x_{i+1} - x_{i}) \vert \text{ for } i=0,\ldots,n-2.
\end{equation}
which can be rewritten as:
\begin{equation}
\vert x_{i+2} - 2 x_{i+1}+ x_{i} \vert \text{ for } i=0,\ldots,n-2.
\end{equation}
which is equivalent to minimizing the quantity
\begin{equation}
\left( x_{i+2} - 2 x_{i+1}+ x_{i} \right)^2 \text{ for } i=0,\ldots,n-2.
\end{equation}

In [2]:
###############################
### 3D Target Tracking function
def TargetTracking(n, maxpos):
    ### parameters
    # n : number of instants
    # maxpos : index of maximum position
    # generate target's random positions
    rng = np.random.default_rng()
    pos =  np.zeros(3*n, dtype=int)
    pos[0:3] = rng.integers(low=0, high=maxpos,size=3)
    delta = rng.integers(low=-1, high=2, size=[3,n]) # x,y,z-displacements
    #print("\nDeltas:\n", delta)
    for i in range(n-1):
        pos[i+1] = np.min( [np.max( [pos[i] + delta[0,i+1], 0]), maxpos] )
        pos[i+1+n] = np.min( [np.max( [pos[i+n] + delta[1,i+1], 0]), maxpos] )
        pos[i+1+2*n] = np.min( [np.max( [pos[i+2*n] + delta[2,i+1], 5]), maxpos] )
    #print("\nTarget positions (x):\n", pos[0:n])
    #print("\nTarget positions (y):\n", pos[n:2*n])
    #print("\nTarget positions (z):\n", pos[2*n:3*n])
        # minimum position
    vmin = 0
    # maximum position
    vmax = 10
    C1 = Bounds(np.array(np.zeros(3*n))+vmin, np.array(np.zeros(3*n))+vmax)
    #print(C1)
    smin = 1 # chaser-target minimum distance
    smax = 2 # chaser-target maximum distance
    c = smin * np.ones(n)
    #print(c)
    d = smax * np.ones(n)
    con = lambda x :[math.sqrt( (x[i]-pos[i])**2+(x[i+n]-pos[i+n])**2+(x[i+2*n]-pos[i+2*n])**2 ) for i in range(n)]
    #print (d)
    C2 = NonlinearConstraint(con, c, d)
    #print(C2)
    f = lambda x : sum([ abs(x[i+2]-2*x[i+1]+x[i] + x[i+2+n]-2*x[i+1+n]+x[i+n] + x[i+2+2*n]-2*x[i+1+2*n]+x[i+2*n]) for i in range(n-2)])
    start = timer()
    sol = opt.minimize(f, np.zeros(3*n), bounds=C1, constraints=(C2,))
    end = timer()
    runtime = end - start
    #print("Runtime (s): ", f'{runtime:9.4f}', "\n")
    # print(np.round(sol.x))
    x = sol.x
    # validation
    # print([math.sqrt( (x[i]-pos[i])**2+(x[i+n]-pos[i+n])**2+(x[i+2*n]-pos[i+2*n])**2) for i in range(n)])
    return runtime

In [ ]:
################
### Main program

maxpos = 10

# runtime = TargetTracking(n, maxpos)
# print("Runtime (s): ", f'{runtime:9.4f}', "\n")

numaverages = 10
numsamples = 10
RTaverages = np.zeros(numaverages, dtype=float)
for i in range(numaverages):
    T = 0
    print("i: ", i, "\n")
    for j in range(numsamples):
        RT = TargetTracking((i + 1) * 100, maxpos)
        print(RT, " ")
        T = T + RT
    print("\n")
    RTaverages[i] = T / numsamples
    print("Average: ", RTaverages[i], "\n")
print("Averages: ", RTaverages, "\n")

i:  0 

16.527743700000002  
17.103310749000002  
16.401153494  
16.64637464499998  
16.72596089000001  
16.32496523200001  
16.444352444000003  
16.76032208999999  
17.79908537  
16.895830444000012  


Average:  16.7629099058 

i:  1 

100.55113033099997  
101.99184734399995  
103.39291746399999  
102.03832805900004  
101.788133991  
101.50723764999998  
104.63937119399998  
101.06950701799997  
103.00000580799997  
104.06715668399988  


Average:  102.40456355429997 

i:  2 

315.91783386299994  
315.58303845600017  
311.561151635  
314.7404536670001  
314.01627915100016  
313.2410824950002  
310.0567629970001  
307.7261469079999  
324.4356787439997  
322.56122711199987  


Average:  314.98396550280006 

i:  3 

748.7533462660003  
754.3083918570001  
755.5214846750005  
754.1093404629992  
762.34387771  
772.122680651999  
773.5168428039997  
773.4504516880006  
772.5488392930001  
835.6380418789995  


Average:  770.2313297286998 

i:  4 

1676.3255898450006  
1690.6311318919998  
